In [123]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/
!ls /home/kesci/input/gzlt/train_set/
!ls /home/kesci/input/gzlt/train_set/201708n/
!ls /home/kesci/input/gzlt/train_set/201708q/
!ls /home/kesci/input/gzlt/test_set/
!ls /home/kesci/input/gzlt/test_set/201808/

gzlt
201708n  201708q  weather_data_2017
201708n1.txt  201708n3.txt  201708n6.txt
201708n2.txt  201708n4.txt  201708n7.txt
201708q1.txt  201708q3.txt  201708q6.txt
201708q2.txt  201708q4.txt  201708q7.txt
201808	weather_data_2018
2018_1.txt  2018_2.txt	2018_3.txt  2018_4.txt	2018_6.txt  2018_7.txt
time: 4.74 s


In [124]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

kesci_submit	  n_tr.csv	    q_tr_quchong.csv  te_quchong.csv
lost+found	  n_tr_quchong.csv  te.csv	      test66.csv
mysubmission.csv  q66.csv	    te_itft.csv       tr_itft.csv
n66.csv		  q_tr.csv	    te_itft_rd.csv    tr_itft_rd.csv
time: 790 ms


In [125]:
# 查看当前kernerl下的package
#!pip list --format=columns

time: 457 µs


In [126]:
# 显示cell运行时长
%load_ext klab-autotime

The klab-autotime extension is already loaded. To reload it, use:
  %reload_ext klab-autotime
time: 8.63 ms


In [127]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn import metrics

time: 9.02 ms


In [128]:
##定义读取训练集数据函数：
def r_d(f):
    n1= pd.read_csv("/home/kesci/input/gzlt/train_set/201708%s/201708%s1.txt"%(f,f), sep='\t',header=None)
    n1.columns=['账期','id','出账收入','labe']
    n2= pd.read_csv("/home/kesci/input/gzlt/train_set/201708%s/201708%s2.txt"%(f,f), sep='\t',header=None)
    #print(n2.shape,'************\n',n2[:5])
    n2.columns=['id','手机品牌','终端型号','首次使用时间','末次使用时间','labe']
    n3= pd.read_csv("/home/kesci/input/gzlt/train_set/201708%s/201708%s3.txt"%(f,f), sep='\t',header=None)
    #print(n3.shape,'************\n',n3[:5])
    n3.columns=['账期','id','联络圈规模','是否出省','是否出境','labe']
    n4= pd.read_csv("/home/kesci/input/gzlt/train_set/201708%s/201708%s4.txt"%(f,f), sep='\t',header=None)
    #print(n4.shape,'************\n',n4[:5])
    n4.columns=['账期','id','漫出省份','labe']
    #n5= pd.read_csv("/home/kesci/input/gzlt/train_set/201708%s/201708%s5.txt"%(f,f), sep='\t',header=None)
    #print(n5.shape,'************\n',n5[:5])
    #n5.columns=['账期','id','是否去过黔东南目标景区','labe']
    n6= pd.read_csv("/home/kesci/input/gzlt/train_set/201708%s/201708%s6.txt"%(f,f), sep='\t',header=None)
    #print(n6.shape,'************\n',n6[:5])
    n6.columns=['日期','时段','id','经度','纬度','labe']
    if f=='n':
        n7= pd.read_csv("/home/kesci/input/gzlt/train_set/201708%s/201708%s7.txt"%(f,f), sep='\t',header=None)
        #print(n7.shape,'************\n',n7[:5])
        n7.columns=['账期','id','APP名称','流量']
    else:
        n7= pd.read_csv("/home/kesci/input/gzlt/train_set/201708%s/201708%s7.txt"%(f,f), sep='\t',header=None)
        #print(n7.shape,'************\n',n7[:5])
        n7.columns=['账期','id','APP名称','流量','labe']   
    uid=pd.DataFrame(list(set(list(n1['id'])+list(n2['id'])+list(n3['id'])+list(n4['id'])+\
        list(n6['id'])+list(n7['id']))),columns=['id'])
    return n1,n2,n3,n4,n6,n7,uid

time: 4.19 ms


In [129]:
'''测试集直接读取：'''
test1= pd.read_csv("/home/kesci/input/gzlt/test_set/201808/2018_1.txt", sep='\t',header=None)
#print(test1.shape,'************\n',test1[:5])
test1.columns=['账期','id','出账收入']
test2= pd.read_csv("/home/kesci/input/gzlt/test_set/201808/2018_2.txt", sep='\t',header=None)
#print(test2.shape,'************\n',test2[:5])
test2.columns=['id','手机品牌','终端型号','首次使用时间','末次使用时间']
test3= pd.read_csv("/home/kesci/input/gzlt/test_set/201808/2018_3.txt", sep='\t',header=None)
#print(test3.shape,'************\n',test3[:5])
test3.columns=['账期','id','联络圈规模','是否出省','是否出境']
test4= pd.read_csv("/home/kesci/input/gzlt/test_set/201808/2018_4.txt", sep='\t',header=None)
#print(test4.shape,'************\n',test4[:5])
test4.columns=['账期','id','漫出省份']
#test5= pd.read_csv("/home/kesci/input/gzlt/test_set/201808/2018_5.txt", sep='\t',header=None)
#print(test5.shape,'************\n',test5[:5])
#test5.columns=['账期','id','是否去过黔东南目标景区']
test6= pd.read_csv("/home/kesci/input/gzlt/test_set/201808/2018_6.txt", sep='\t',header=None)
#print(test6.shape,'************\n',test6[:5])
test6.columns=['日期','时段','id','经度','纬度']
test7= pd.read_csv("/home/kesci/input/gzlt/test_set/201808/2018_7.txt", sep='\t',header=None)
#print(test7.shape,'************\n',test7[:5])
test7.columns=['账期','id','APP名称','流量']
test_uid=pd.DataFrame(list(set(list(test1['id'])+list(test2['id'])+list(test3['id'])+list(test4['id'])+\
        list(test6['id'])+list(test7['id']))),columns=['id'])
print(test_uid.shape)

(50200, 1)
time: 20 s


In [130]:
##表1特征提取：
def f1(d):
    '''每月的出账收入'''
    d=d.groupby(['id','账期'],as_index=False)['出账收入'].agg({'出账收入':'mean'})
    riqi=list(set(d['账期']))
    nd=pd.DataFrame(list(set(d['id'])),columns=['id'])
    for i in riqi:
        m=d[d['账期']==i][['id','出账收入']]
        m.columns=['id',str(i)[-1]+'出账收入']
        nd=pd.merge(nd,m,how='left',on=['id'])
    del riqi,m,d
    return nd

time: 1.79 ms


In [131]:
##读取训练数据：
n1,n2,n3,n4,n6,n7,n_uid=r_d(f='n')
q1,q2,q3,q4,q6,q7,q_uid=r_d('q')
print(len(n_uid))


93400
time: 44.2 s


In [135]:
##表2提取特征：
import time
def f2(d):
    d=d.fillna(-99)
    #nd=pd.DataFrame(list(set(d['id'])),columns=['id'])
    m=d.groupby(['id'],as_index=False)['id'].agg({'id_count_2':'count'})
    m1=d[d['手机品牌']==-99].groupby(['id'],as_index=False)['手机品牌'].agg({'手机品牌_2':'count'})
    m=pd.merge(m,m1,how='left',on=['id'])#.fillna(0)
    m['手机品牌缺失率_2']=m['手机品牌_2']/m['id_count_2']
    
    m1=d[d['手机品牌']!=-99].groupby(['id'],as_index=False)['手机品牌'].agg({'手机品牌nunique_2':'nunique'})
    m=pd.merge(m,m1,how='left',on=['id'])
    
    m1=d[d['终端型号']==-99].groupby(['id'],as_index=False)['终端型号'].agg({'终端型号count_2':'count'})
    m=pd.merge(m,m1,how='left',on=['id'])
    m['终端型缺失率_2']=m['终端型号count_2']/m['id_count_2']
    
    m1=d[d['终端型号']!=-99].groupby(['id'],as_index=False)['终端型号'].agg({'终端型号nunique_2':'nunique'})
    m=pd.merge(m,m1,how='left',on=['id'])
    
    d['末次使用时间']=d['末次使用时间'].apply(lambda x:\
    int(time.mktime(time.strptime(str(int(x)), "%Y%m%d%H%M%S"))) if int(x)>0 else -99)
    d['首次使用时间']=d['首次使用时间'].apply(lambda x:\
    int(time.mktime(time.strptime(str(int(x)), "%Y%m%d%H%M%S"))) if int(x)>0 else -99)
    d['末-首次使用时间']=(d['末次使用时间']-d['首次使用时间'])/3600
    
    m1=d.groupby(['id'],as_index=False)['末-首次使用时间'].agg({'末-首次使用时间mean':'mean',\
        '末-首次使用时间max':'max',\
        '末-首次使用时间min':'min',\
        '末-首次使用时间std':'std',\
         })
    m=pd.merge(m,m1,how='left',on=['id'])
    del m1
    ##加小时，周期特征：
    d['末次month']=d['末次使用时间'].apply(lambda x:\
    int(str(x)[4:6]) if int(x)>0 else -99)
    d['首次month']=d['首次使用时间'].apply(lambda x:\
    int(str(x)[4:6]) if int(x)>0 else -99)
    m1=d[d['末次month']!=-99].groupby(['id'],as_index=False)['末次month'].agg({'末次monthmean':'mean',\
        '末次month间max':'max',\
        '末次monthmin':'min',\
        '末次monthstd':'std',\
         })
    m=pd.merge(m,m1,how='left',on=['id'])
    m1=d[d['首次month']!=-99].groupby(['id'],as_index=False)['首次month'].agg({'首次monthmean':'mean',\
        '末次month间max':'max',\
        '末次monthmin':'min',\
        '首次monthstd':'std',\
         })
    m=pd.merge(m,m1,how='left',on=['id'])
    
    m1=d[d['末次month']==-99].groupby(['id'],as_index=False)['末次month'].agg({'末次month99':'count'})
    m=pd.merge(m,m1,how='left',on=['id'])
    m1=d[d['首次month']==-99].groupby(['id'],as_index=False)['首次month'].agg({'首次month99':'count'})
    m=pd.merge(m,m1,how='left',on=['id'])
    
    del m1
    
    return m

(289203, 7) 
******************
 id          5600
手机品牌         750
终端型号        5290
首次使用时间    285263
末次使用时间    277300
labe           1
y              4
dtype: int64
time: 241 ms


In [136]:
##表3提取特征：
def f3(d):
    m=pd.DataFrame(list(set(d['id'])),columns=['id'])
    m1=d[d['联络圈规模']!=-1].groupby(['id'],as_index=False)['联络圈规模'].agg({'联络圈规模_3':'mean'})
    m=pd.merge(m,m1,how='left',on=['id']).fillna(-1)
    
    m1=d.groupby(['id','账期'],as_index=False)['是否出省'].agg({'是否出省':'mean'})
    for i in list(set(m1['账期'])):
        mm=m1[m1['账期']==i][['id','是否出省']]
        mm.columns=['id',str(i)[-1]+'是否出省']
        m=pd.merge(m,mm,how='left',on=['id'])
    m1=d.groupby(['id','账期'],as_index=False)['是否出境'].agg({'是否出境':'mean'})
    for i in list(set(m1['账期'])):
        mm=m1[m1['账期']==i][['id','是否出境']]
        mm.columns=['id',str(i)[-1]+'是否出境']
        m=pd.merge(m,mm,how='left',on=['id'])
    del m1
    return m

time: 3.53 ms


In [137]:
##表4提取特征：
def f4(d):
    m=d.groupby(['id'],as_index=False)['漫出省份'].agg({'漫出省份count_4':'count',\
       '漫出省份nunique_4':'nunique'
    })
    return m

(36499, 4) 
******************
 账期          2
id      12958
漫出省份       31
labe        1
dtype: int64
          账期                id 漫出省份  labe
4306  201706  9987406611703100   广西     1
4305  201706  9987406611703100   广西     1
4304  201706  9987406611703100   广西     1
4307  201706  9987406611703100   广西     1
4308  201706  9987406611703100   广西     1
4309  201706  9987406611703100   广东     1
4303  201706  9987406611703100   广东     1
4302  201706  9987406611703100   广东     1
3048  201707  9986868023697813   浙江     1
3052  201707  9986868023697813   江西     1
[nan, '西藏', '陕西', '江苏', '四川', '北京', '天津', '吉林', '青海', '河北', '辽宁', '广西', '新疆', '宁夏', '甘肃', '重庆', '广东', '山西', '山东', '黑龙江', '安徽', '浙江', '河南', '内蒙古', '福建', '贵州', '海南', '湖南', '湖北', '上海', '江西', '云南']
(400, 4)
time: 32.7 ms


In [138]:
##表5提取特征：
def f5(d):
    m=pd.DataFrame(list(set(d['id'])),columns=['id'])
    for month in list(set(d['账期'])):
        m1=d[d['账期']==month][['id','是否去过黔东南目标景区']]
        m1.columns=['id',str(month)[-1]+'是否去过黔东南目标景区']
        m=pd.merge(m,m1,how='left',on=['id'])
    del m1
    return m

time: 1.42 ms


In [140]:
##表6提取特征：
def f6_1(d):
    d['时间戳']=d['日期'].apply(lambda x:\
    int(time.mktime(time.strptime(x, "%Y-%m-%d"))))
    d['日期_m']=d['日期'].apply(lambda x:int(str(x).split('-')[1]))
    m=d.groupby(['id'],as_index=False)['日期'].agg({\
       '日期nunique_6':'nunique',\
       '日期count_6':'count'
    })
    m['日期count_6/日期nunique_6']=m['日期count_6']/(1+m['日期nunique_6'])
    m=m.drop(['日期count_6'],axis=1)
    
    m1=d.groupby(['id','日期_m'],as_index=False)['日期_m'].agg({\
       '日期_m_count':'count'
    })
    for i in list(set(m1['日期_m'])):
        m2=m1[m1['日期_m']==i][['id','日期_m_count']]
        m2.columns=['id','日期_m_count'+str(i)]
        m=m.merge(m2,how='left',on=['id']).fillna(0)
        m['日期_m_count'+str(i)]=m['日期_m_count'+str(i)].apply(lambda x:1 if x>0 else 0)
    del m1,m2
    return m
    
def f6(d):
    d['基站']=d['经度'].astype(str)+'_'+d['纬度'].astype(str)
    m=d.groupby(['id'],as_index=False)['基站'].agg({'基站count_6':'count',\
       '基站nunique_6':'nunique'
    })
    m1=d.groupby(['id'],as_index=False)['经度'].agg({'经度mean_6':'mean',\
       '经度max_6':'max',\
       '经度min_6':'min',\
       '经度std_6':'std',\
       '经度median_6':'median',
       '经度nunique_6':'nunique',
    })
    m=pd.merge(m,m1,how='left',on=['id'])
    del m1
    m1=d.groupby(['id'],as_index=False)['纬度'].agg({'纬度mean_6':'mean',\
       '纬度max_6':'max',\
       '纬度min_6':'min',\
       '纬度std_6':'std',\
       '纬度median_6':'median',
       '纬度nunique_6':'nunique',
    })
    m=pd.merge(m,m1,how='left',on=['id'])
    del m1
    m1=d.groupby(['id'],as_index=False)['时段'].agg({'时段mean_6':'mean',\
       '时段max_6':'max',\
       '时段min_6':'min',\
       '时段std_6':'std',\
       '时段median_6':'median',
       '时段nunique_6':'nunique',
    })
    m=pd.merge(m,m1,how='left',on=['id'])
    del m1
    
    t=[]
    for u in m['id']:
        m1=d[d.id==u][['基站','经度','纬度','日期','时段']]
        m1=m1.sort_values(by=['日期','时段'],ascending=True).reset_index(drop=True)
        inds=[]
        if len(m1)>1:
            for i in range(len(m1)-1):
                if m1.loc[i,'基站']==m1.loc[i+1,'基站']:
                    if m1.loc[i,'日期']==m1.loc[i+1,'日期']:
                        inds.append(i+1)
            m1=m1.drop(inds,axis=0).reset_index(drop=True)
            #print(len(m1))
        
        if len(m1)>0:
            t.append([u,m1['经度'].max(),m1['经度'].min(),m1['经度'].mean(),m1['经度'].std(),\
            m1['纬度'].max(),m1['纬度'].min(),m1['纬度'].mean(),m1['纬度'].std(),\
            len(m1)])
        #else:
        #    t.append([u,0,0,0,0])
               
    t=pd.DataFrame(t)
    t.columns=['id']+['fff{}'.format(i) for i in range(t.shape[1]-1)]
    del m1
    m=pd.merge(m,t,how='left',on=['id'])
    del t
    return m

                 日期   时段                id          经度         纬度  labe  \
2309855  2017-06-13  0.0  5885140327545400  106.737292  26.491528     1   
137727   2017-07-16  0.0  7628503026181759  106.689060  26.577640     1   
695596   2017-07-26  0.0  9795428722591227  106.681080  26.571250     1   
1432309  2017-07-16  0.0  5806773608286300  107.780970  26.196025     1   
2047252  2017-06-22  0.0   891223810895000  106.693860  26.591010     1   
1189054  2017-07-27  0.0  3850423310904900  106.690790  26.562120     1   
695600   2017-07-22  0.0  2474772103093852  106.718840  26.587360     1   
1189058  2017-07-07  0.0  4797083896419762  106.614929  26.688263     1   
435693   2017-07-20  0.0  4256790410176798  105.490640  26.202920     1   
435692   2017-07-19  0.0  3758073107952600  106.724600  26.616190     1   
2597625  2017-06-12  0.0  3846048361566500  106.623568  26.652063     1   
2047260  2017-06-10  0.0  9650368071572749  106.665640  26.586700     1   
2597603  2017-06-18  0.0 

In [142]:
##表7提取特征：
print(q7.sort_values(by=['id'],ascending=True)[:100])
def f7(d):
    m=d.groupby(['id'],as_index=False)['APP名称'].agg({'APP名称count_7':'count',\
       'APP名称nunique_7':'nunique'
    })
    m1=d.groupby(['id'],as_index=False)['流量'].agg({'流量mean_7':'mean',\
       '流量sum_7':'sum',\
       '流量max_7':'max',\
       '流量min_7':'min',\
       '流量std_7':'std',\
    })
    m=pd.merge(m,m1,how='left',on=['id'])
    del m1
    return m
def f7_1(d):
    d['账期_month']=d['账期'].apply(lambda x:str(x)[-2:])
    mm=pd.DataFrame(list(set(d['id'])),columns=['id'])
    m=d.groupby(['id','账期_month'],as_index=False)['APP名称'].agg({'APP名称count_7':'count'})
    for  col in list(set(m['账期_month'])):
        guo=m[m['账期_month']==col][['id','APP名称count_7']]
        guo.columns=['id','APP名称count_7'+col]
        mm=mm.merge(guo,how='left',on=['id']).fillna(0)
        mm['APP名称count_7'+col]=mm['APP名称count_7'+col].apply(lambda x:1 if x>0 else 0)
    
    m=d.groupby(['id','账期_month'],as_index=False)['流量'].agg({'流量_mean_7':'mean'})
    for  col in list(set(m['账期_month'])):
        guo=m[m['账期_month']==col][['id','流量_mean_7']]
        guo.columns=['id','流量_mean_7'+col]
        mm=mm.merge(guo,how='left',on=['id'])#.fillna(0)
    del m,guo
    return mm


(173117, 5) 
******************
 账期           2
id        4717
APP名称      762
流量       11485
labe         1
dtype: int64
            账期              id       APP名称      流量  labe
88662   201707   1448103998000        乐视视频    0.00     1
53088   201707   1448103998000        最美天气    0.01     1
23842   201707   1448103998000     360手机卫士    0.03     1
90240   201707   1448103998000       爱奇艺视频    0.00     1
115177  201706   1448103998000          QQ    0.00     1
154063  201706   1448103998000      腾讯手机管家    0.00     1
74423   201707   1448103998000          微信    0.09     1
8813    201707   1448103998000        优酷视频    0.00     1
56182   201707   1448103998000          QQ    0.01     1
72515   201707   1448103998000       开心消消乐    0.00     1
126719  201706  17398718813730          QQ   10.01     1
9004    201707  17398718813730        美图秀秀    0.67     1
71785   201707  17398718813730        百度地图    1.80     1
12634   201707  17398718813730   掌阅iReader    0.00     1
101515  201706  17398718

In [143]:
##结合特征：q开头的为去旅游的训练集，n开头的为不去旅游的训练集，test开头的为测试集
q_uid['label']=1 ##训练集中去旅游的用户
n_uid['label']=0 ##训练集中不去旅游的用户
q_train=pd.merge(q_uid,f1(q1),how='left',on=['id'])
q_train=pd.merge(q_train,f2(q2),how='left',on=['id'])
q_train=pd.merge(q_train,f3(q3),how='left',on=['id'])
q_train=pd.merge(q_train,f4(q4),how='left',on=['id'])
#q_train=pd.merge(q_train,f5(q5),how='left',on=['id'])
q_train=pd.merge(q_train,f6_1(q6),how='left',on=['id'])
try:
    mm=pd.read_csv('q66.csv')
except:
    mm=f6(q6) ##表6特征提取花时间，保存方便再次使用
    mm.to_csv('q66.csv',index=None)
q_train=pd.merge(q_train,mm,how='left',on=['id'])

q_train=pd.merge(q_train,f7(q7),how='left',on=['id'])
q_train=pd.merge(q_train,f7_1(q7),how='left',on=['id'])

n_train=pd.merge(n_uid,f1(n1),how='left',on=['id'])
n_train=pd.merge(n_train,f2(n2),how='left',on=['id'])
n_train=pd.merge(n_train,f3(n3),how='left',on=['id'])
n_train=pd.merge(n_train,f4(n4),how='left',on=['id'])
#n_train=pd.merge(n_train,f5(n5),how='left',on=['id'])
n_train=pd.merge(n_train,f6_1(n6),how='left',on=['id'])
try:
    mm=pd.read_csv('n66.csv')
except:
    mm=f6(n6)
    mm.to_csv('n66.csv',index=None)
n_train=pd.merge(n_train,mm,how='left',on=['id'])
#n_train=pd.merge(n_train,f6(n6),how='left',on=['id'])
n_train=pd.merge(n_train,f7(n7),how='left',on=['id'])
n_train=pd.merge(n_train,f7_1(n7),how='left',on=['id'])

test=pd.merge(test_uid,f1(test1),how='left',on=['id'])
test=pd.merge(test,f2(test2),how='left',on=['id'])
test=pd.merge(test,f3(test3),how='left',on=['id'])
test=pd.merge(test,f4(test4),how='left',on=['id'])
#test=pd.merge(test,f5(test5),how='left',on=['id'])
test=pd.merge(test,f6_1(test6),how='left',on=['id'])
try:
    mm=pd.read_csv('test66.csv')
except:
    mm=f6(test6)
    mm.to_csv('test66.csv',index=None)
test=pd.merge(test,mm,how='left',on=['id'])
#test=pd.merge(test,f6(test6),how='left',on=['id'])
test=pd.merge(test,f7(test7),how='left',on=['id'])
test=pd.merge(test,f7_1(test7),how='left',on=['id'])

print(q_train.shape,n_train.shape,test.shape)

(5600, 78) (93400, 78) (50200, 77)
time: 23min 4s


In [144]:
train=pd.concat([q_train,n_train],axis=0).reset_index(drop=True)

#train=train[train['id']>10000000000000].reset_index(drop=True)
print(train[train['id']>10000000000000]['label'].value_counts())
label=train['label']
print(train['label'].value_counts())
print(train['id'].min(),train['id'].max())
print('训练集中大于m的个数',len(train[train['id']>10000000000000]))
print('训练集且去旅游中大于m的个数',len(q_train[q_train['id']>10000000000000]))
print(train[train['id']>10000000000000]['id'].min())
print(test['id'].min(),test['id'].max())
print('测试集中小于m的个数',len(test[test['id']<10000000000000]))

xiu=[i for i in test['id'].values if int(i) in train['id'].values]
print('相同用户个数：',len(xiu))
xiu2=train[train.id.isin(xiu)][['id','label']].reset_index(drop=True)
print(len(xiu2))
print('相同用户标签分布：',xiu2['label'].value_counts())
#import matplotlib.pyplot as plt
print(train.columns)
print(test.columns)
nn=test[['id']]
nn['label']=-1
dd=pd.concat([train[train['id']>10000000000000][['id','label']],nn[['id','label']]],axis=0).reset_index(drop=True)
dd=dd.sort_values(by='id')
#print(dd[:1000])

0    93390
1     5599
Name: label, dtype: int64
0    93400
1     5600
Name: label, dtype: int64
1009387204000 9999717474837900
训练集中大于m的个数 98989
训练集且去旅游中大于m的个数 5599
10204443360000
595941207920 9999920327264900
测试集中小于m的个数 5
相同用户个数： 0
0
相同用户标签分布： Series([], Name: label, dtype: int64)
Index(['id', 'label', '6出账收入', '7出账收入', 'id_count_2', '手机品牌_2', '手机品牌缺失率_2',
       '手机品牌nunique_2', '终端型号count_2', '终端型缺失率_2', '终端型号nunique_2',
       '末-首次使用时间mean', '末-首次使用时间max', '末-首次使用时间min', '末-首次使用时间std',
       '末次monthmean', '末次month间max_x', '末次monthmin_x', '末次monthstd',
       '首次monthmean', '末次month间max_y', '末次monthmin_y', '首次monthstd',
       '末次month99', '首次month99', '联络圈规模_3', '6是否出省', '7是否出省', '6是否出境', '7是否出境',
       '联络圈规模_3_1', '漫出省份count_4', '漫出省份nunique_4', '日期nunique_6',
       '日期count_6/日期nunique_6', '日期_m_count5', '日期_m_count6', '日期_m_count7',
       '基站count_6', '基站nunique_6', '经度mean_6', '经度max_6', '经度min_6', '经度std_6',
       '经度median_6', '经度nunique_6', '纬度mean_6', '纬度max_6', '纬度m

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [146]:
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import roc_auc_score
import xgboost as xgb
def xgb_model(new_train,y,new_test,lr,N):
  '''定义模型'''
  xgb_params = {'booster': 'gbtree',
          'eta':lr, 'max_depth': 5, 'subsample': 0.8, 'colsample_bytree': 0.8, 
          'objective':'binary:logistic',
          'eval_metric': 'auc',
          'silent': True,
          }
  #skf=StratifiedKFold(y,n_folds=5,shuffle=True,random_state=2018)
  skf=StratifiedKFold(n_splits=N,shuffle=True,random_state=42)
  oof_xgb=np.zeros(new_train.shape[0])
  prediction_xgb=np.zeros(new_test.shape[0])
  for i,(tr,va) in enumerate(skf.split(new_train,y)):
    print('fold:',i+1,'training')
    dtrain = xgb.DMatrix(new_train[tr],y[tr])
    dvalid = xgb.DMatrix(new_train[va],y[va])
    watchlist = [(dtrain, 'train'), (dvalid, 'valid_data')]
    bst = xgb.train(dtrain=dtrain, num_boost_round=30000, evals=watchlist, early_stopping_rounds=200, \
    verbose_eval=400, params=xgb_params)
    oof_xgb[va] += bst.predict(xgb.DMatrix(new_train[va]), ntree_limit=bst.best_ntree_limit)
    prediction_xgb += bst.predict(xgb.DMatrix(new_test), ntree_limit=bst.best_ntree_limit)
  print('the roc_auc_score for train:',roc_auc_score(y,oof_xgb))
  prediction_xgb/=N
  return oof_xgb,prediction_xgb
def lgb_model(new_train,y,new_test):
    params = {
    'learning_rate': 0.01,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'num_leaves': 1000,
    'verbose': -1,
    'max_depth': -1,
  #  'reg_alpha':2.2,
  #  'reg_lambda':1.4,
    'seed':42,
    }
    #skf=StratifiedKFold(y,n_folds=5,shuffle=True,random_state=2018)
    skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
    oof_lgb=np.zeros(new_train.shape[0])
    prediction_lgb=np.zeros(new_test.shape[0])
    feature_importance_df = pd.DataFrame()
    for i,(tr,va) in enumerate(skf.split(new_train,y)):
        print('fold:',i+1,'training')
        dtrain = lgb.Dataset(new_train[tr],y[tr])
        dvalid = lgb.Dataset(new_train[va],y[va],reference=dtrain)
        bst = lgb.train(params, dtrain, num_boost_round=30000, valid_sets=dvalid, verbose_eval=400,early_stopping_rounds=200)
        oof_lgb[va] += bst.predict(new_train[va], num_iteration=bst.best_iteration)
        prediction_lgb += bst.predict(new_test, num_iteration=bst.best_iteration)
        '''
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = list(new_train.columns)
        fold_importance_df["importance"] = bst.feature_importance(importance_type='split', iteration=bst.best_iteration)
        fold_importance_df["fold"] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        '''
    
    print('the roc_auc_score for train:',roc_auc_score(y,oof_lgb))
    prediction_lgb/=5
    return oof_lgb,prediction_lgb,feature_importance_df
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
def model(train_,label,test_):
    #模型训练：
    n_folds=5
    stack_train = np.zeros(train_.shape[0])
    stack_test = np.zeros(test_.shape[0])
    skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=2019)
    for i, (tr, va) in enumerate(skf.split(train_,label)):
        print('stack:%d/%d' % ((i + 1), n_folds))
        #cfc = LinearSVC(random_state=2019)
        cfc= LogisticRegression(random_state=2019,C=8)
        cfc.fit(train_[tr], label[tr])
        #score_va = cfc._predict_proba_lr(train_[va])[:,1]
        #score_te = cfc._predict_proba_lr(test_)[:,1]
        score_va = cfc.predict_proba(train_[va])[:,1]
        score_te = cfc.predict_proba(test_)[:,1]
        print("per model roc_auc_score:", metrics.roc_auc_score(label[va],score_va))
        stack_train[va] += score_va
        stack_test += score_te
    print("model roc_auc_score:",roc_auc_score(label,stack_train))
    stack_test /= n_folds
    return stack_train,stack_test

time: 7.09 ms


In [147]:
from sklearn.utils import shuffle
new_tr=train.drop(['label'],axis=1)
new_te=test[new_tr.columns]

time: 60.1 ms


词向量部分从这里开始...

In [155]:
##按时间，构造每个用户的基站序列：基站直接由经纬度表示
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import preprocessing
import gc
def one_hot_col(col):
    '''标签编码'''
    lbl = preprocessing.LabelEncoder()
    lbl.fit(col)
    return lbl
gc.collect()
'''
try:
    n_tr=pd.read_csv('n_tr.csv') ##训练集中不去旅游用户的基站序列
    q_tr=pd.read_csv('q_tr.csv') ##训练集中去旅游用户的基站序列
    te=pd.read_csv('te.csv') ##测试集中用户的基站序列
except:
    n6=n6[['id','基站']]
    q6=q6[['id','基站']]
    test6=test6[['id','基站']]

    seq=list(set(list(set(n6['基站']))+list(set(q6['基站']))+list(set(test6['基站'])))) ##训练集和测试集中所有基站总个数
    print('总基站个数：',len(seq))
    
    ##对基站做变换：
    mm={}
    for i in range(len(seq)):
        mm[seq[i]]=i
    ##对训练集，测试集中的基站进行映射：
    n6['基站']=n6['基站'].map(mm)
    q6['基站']=q6['基站'].map(mm)
    test6['基站']=test6['基站'].map(mm)

    ##映射后的训练集中不去旅游的序列：
    m={}
    for i,row in n6.iterrows():
        m.setdefault(row['id'],[]).append(row['基站'])
    for k, v in m.items():
        m[k]=' '.join([str(i) for i in v])
    m=pd.DataFrame.from_dict(m,orient='index')
    m=m.reset_index()
    m.columns=['id','基站']
    m.to_csv('n_tr.csv',index=None)
    print('n_tr训练集构造完毕！')
    
    ##训练集去旅游的序列：
    m={}
    for i,row in q6.iterrows():
        m.setdefault(row['id'],[]).append(row['基站'])
    for k, v in m.items():
        m[k]=' '.join([str(i) for i in v])
    m=pd.DataFrame.from_dict(m,orient='index')
    m=m.reset_index()
    m.columns=['id','基站']
    m.to_csv('q_tr.csv',index=None)
    print('q_tr训练集构造完毕！')
    
    ##测试集的序列：
    gc.collect()
    m={}
    for i,row in test6.iterrows():
        m.setdefault(row['id'],[]).append(row['基站'])
    for k, v in m.items():
        m[k]=' '.join([str(i) for i in v])
    m=pd.DataFrame.from_dict(m,orient='index')
    m=m.reset_index()
    m.columns=['id','基站']
    m.to_csv('te.csv',index=None)
    print('te测试集构造完毕！')
    del m
try:
    n_tr=pd.read_csv('n_tr.csv')
    q_tr=pd.read_csv('q_tr.csv')
    te=pd.read_csv('te.csv')
except:
    print('数据不存在！')
'''

time: 9.61 s


In [157]:
##和上面的一模一样，构造用户基站序列。这里根据日期，对一天中，连续两个相同的基站去重。（直接跑就行，不用管）
from tqdm import *
'''
try:
    n_tr_quchong=pd.read_csv('n_tr_quchong.csv')
    q_tr_quchong=pd.read_csv('q_tr_quchong.csv')
    te_quchong=pd.read_csv('te_quchong.csv')
except:
    n1,n2,n3,n4,n6,n7,n_uid=r_d(f='n')
    q1,q2,q3,q4,q6,q7,q_uid=r_d('q')
    test6= pd.read_csv("/home/kesci/input/gzlt/test_set/201808/2018_6.txt", sep='\t',header=None)
    test6.columns=['日期','时段','id','经度','纬度']

    n6['基站']=n6['经度'].astype(str)+'_'+n6['纬度'].astype(str)
    q6['基站']=q6['经度'].astype(str)+'_'+q6['纬度'].astype(str)
    test6['基站']=test6['经度'].astype(str)+'_'+test6['纬度'].astype(str)
    n6=n6[['id','基站','日期']]
    q6=q6[['id','基站','日期']]
    test6=test6[['id','基站','日期']]
    
    ##先对数据集去重：
    delindex=[]
    for i in tqdm(range(len(n6)-1)):
        if n6.loc[i,'id']==n6.loc[i+1,'id']:
            if n6.loc[i,'日期']==n6.loc[i+1,'日期'] and n6.loc[i,'基站']==n6.loc[i+1,'基站']:
                delindex.append(i)
    n6_quchong=n6.drop(delindex,axis=0).reset_index(drop=True)
    
    delindex=[]
    for i in tqdm(range(len(q6)-1)):
        if q6.loc[i,'id']==q6.loc[i+1,'id']:
            if q6.loc[i,'日期']==q6.loc[i+1,'日期'] and q6.loc[i,'基站']==q6.loc[i+1,'基站']:
                delindex.append(i)
    q6_quchong=q6.drop(delindex,axis=0).reset_index(drop=True)
    
    delindex=[]
    for i in tqdm(range(len(test6)-1)):
        if test6.loc[i,'id']==test6.loc[i+1,'id']:
            if test6.loc[i,'日期']==test6.loc[i+1,'日期'] and test6.loc[i,'基站']==test6.loc[i+1,'基站']:
                delindex.append(i)
    test6_quchong=test6.drop(delindex,axis=0).reset_index(drop=True)
    
    
    seq=list(set(list(set(n6_quchong['基站']))+list(set(q6_quchong['基站']))+list(set(test6_quchong['基站']))))
    print('总基站个数：',len(seq))
    mm={}
    for i in range(len(seq)):
        mm[seq[i]]=i
    ##对基站进行映射：
    n6_quchong['基站']=n6_quchong['基站'].map(mm)
    q6_quchong['基站']=q6_quchong['基站'].map(mm)
    test6_quchong['基站']=test6_quchong['基站'].map(mm)

    ##训练集不去旅游的序列：
    
    m={}
    for i,row in n6_quchong.iterrows():
        m.setdefault(row['id'],[]).append(row['基站'])
    for k, v in m.items():
        m[k]=' '.join([str(i) for i in v])
    m=pd.DataFrame.from_dict(m,orient='index')
    m=m.reset_index()
    m.columns=['id','基站']
    m.to_csv('n_tr_quchong.csv',index=None)
    print('n_tr_quchong训练集构造完毕！')
    
    ##训练集去旅游的序列：
    m={}
    for i,row in q6_quchong.iterrows():
        m.setdefault(row['id'],[]).append(row['基站'])
    for k, v in m.items():
        m[k]=' '.join([str(i) for i in v])
    m=pd.DataFrame.from_dict(m,orient='index')
    m=m.reset_index()
    m.columns=['id','基站']
    m.to_csv('q_tr_quchong.csv',index=None)
    print('q_tr_quchong训练集构造完毕！')
    
    ##测试集的序列：
    gc.collect()
    m={}
    for i,row in test6_quchong.iterrows():
        m.setdefault(row['id'],[]).append(row['基站'])
    for k, v in m.items():
        m[k]=' '.join([str(i) for i in v])
    m=pd.DataFrame.from_dict(m,orient='index')
    m=m.reset_index()
    m.columns=['id','基站']
    m.to_csv('te_quchong.csv',index=None)
    print('te_quchong测试集构造完毕！')
    del m
try:
    n_tr_quchong=pd.read_csv('n_tr_quchong.csv')
    q_tr_quchong=pd.read_csv('q_tr_quchong.csv')
    te_quchong=pd.read_csv('te_quchong.csv')
except:
    print('数据不存在！')
'''

time: 9.09 s


In [159]:
'''
##用TfidfVectorizer训练模型：基站去重部分
print(n_tr_quchong.shape,q_tr_quchong.shape,te_quchong.shape)
from sklearn.linear_model import RidgeClassifier
try:
    te_itft_rd=pd.read_csv('te_itft_rd.csv')
    tr_itft_rd=pd.read_csv('tr_itft_rd.csv')
except:
    gc.collect()
    tr=pd.concat([n_tr_quchong,q_tr_quchong,te_quchong],axis=0).reset_index(drop=True)

    vec = TfidfVectorizer(ngram_range=(1,1),min_df=1, max_df=0.9,use_idf=1,smooth_idf=1,\
                      sublinear_tf=1)

    term_doc = vec.fit_transform(tr['基站'])
    trn_term_doc =term_doc[:len(n_tr_quchong)+len(q_tr_quchong)]
    test_term_doc =term_doc[len(n_tr_quchong)+len(q_tr_quchong):]
    del term_doc
    print(trn_term_doc.shape,test_term_doc.shape)
    
    y=pd.Series([0]*n_tr_quchong.shape[0]+[1]*q_tr_quchong.shape[0])

    oof_xgb,prediction_xgb=xgb_model(trn_term_doc,pd.Series(y),test_term_doc,0.01,5)
    ##保存特征：
    gc.collect()
    te_quchong=te_quchong.drop(['基站'],axis=1)
    te_quchong['itft_rd']=prediction_xgb
    te_quchong.to_csv('te_itft_rd.csv',index=None)
    del te_quchong
    gc.collect()
    tr=tr.drop(['基站'],axis=1)
    tr=tr[:len(n_tr)+len(q_tr)]
    tr['itft_rd']=oof_xgb
    tr.to_csv('tr_itft_rd.csv',index=None)
    del tr
    gc.collect()
    #oof_lgb,prediction_lgb,feature_importance_df=lgb_model(trn_term_doc,pd.Series(y),test_term_doc)
try:
    te_itft_rd=pd.read_csv('te_itft_rd.csv')
    tr_itft_rd=pd.read_csv('tr_itft_rd.csv')
except:
    print('shujubucunzai!')
'''

(76346, 2) (5575, 2) (41848, 2)
time: 151 ms


In [160]:
'''
##用TfidfVectorizer训练模型：基站不去重部分
try:
    te_itft=pd.read_csv('te_itft.csv')
    tr_itft=pd.read_csv('tr_itft.csv')
except:
    gc.collect()
    tr=pd.concat([n_tr,q_tr,te],axis=0).reset_index(drop=True)

    vec = TfidfVectorizer(ngram_range=(1,1),min_df=1, max_df=0.9,use_idf=1,smooth_idf=1,\
                      sublinear_tf=1)

    term_doc = vec.fit_transform(tr['基站'])
    trn_term_doc =term_doc[:len(n_tr)+len(q_tr)]
    test_term_doc =term_doc[len(n_tr)+len(q_tr):]
    del term_doc
    print(trn_term_doc.shape,test_term_doc.shape)
    
    y=[0]*n_tr.shape[0]+[1]*q_tr.shape[0]
    oof_xgb,prediction_xgb=xgb_model(trn_term_doc,pd.Series(y),test_term_doc,0.01,5)
    
    ##保存特征：
    gc.collect()
    te=te.drop(['基站'],axis=1)
    te['itft_xgb']=prediction_xgb
    te.to_csv('te_itft.csv',index=None)
    del te
    gc.collect()
    tr=tr.drop(['基站'],axis=1)
    tr=tr[:len(n_tr)+len(q_tr)]
    tr['itft_xgb']=oof_xgb
    tr.to_csv('tr_itft.csv',index=None)
    del tr
    gc.collect()

try:
    te_itft=pd.read_csv('te_itft.csv')
    tr_itft=pd.read_csv('tr_itft.csv')
except:
    print('shujubucunzai!')
'''

time: 147 ms


In [163]:
'''
##结合词向量特征：
print(pd.merge(new_te,te_itft,how='left',on=['id']).shape)
##结合itft特征（基站没去重）：
new_tr=pd.merge(new_tr,tr_itft,how='left',on=['id'])
new_te=pd.merge(new_te,te_itft,how='left',on=['id'])
##结合itft特征（基站去重）：
new_tr=pd.merge(new_tr,tr_itft_rd,how='left',on=['id'])
new_te=pd.merge(new_te,te_itft_rd,how='left',on=['id'])
'''

(50200, 78)
time: 248 ms


In [190]:
##最终模型训练：
my_xx=new_tr.copy() ##训练集
my_xx['label']=label
my_te=new_te.copy() ##测试集

useful_id_tr4=list(set(q4['id']))+list(set(n4['id'])) ##训练集中，在表4的所有用户
useful_id_tr7=list(set(n7['id'])) ##训练集中，在表7的不去旅游的用户
useful_id_tr2=list(set(n2['id'])) ##训练集中，在表2的不去旅游的用户

useful_id_tr=list(set(q6['id']))+list(set(n6['id']))##训练集中，在表6的所有用户
#useful_id_tr=list(set(n6['id']))
useful_id_te=list(set(test6['id']))##测试集中，在表6的所有用户
useful_id_tr1=[i for i in list(set(n1['id'])) if i not in useful_id_tr2] ##训练集中，不在表2的，且不去旅游的用户：25个

my_xx1=my_xx[(my_xx.id.isin(useful_id_tr))&\  ##只取在表6的用户训练
(~my_xx.id.isin(useful_id_tr1))  ##不去旅游中，去掉不再表2的25个用户
].reset_index(drop=True)

##模型训练：
oof_xgb1,prediction_xgb1=xgb_model(np.array(my_xx1.drop(['id','label'],axis=1)),\
                       my_xx1['label'].values,\
                       np.array(my_te.drop(['id'],axis=1)),0.01,5)
                       
from sklearn import metrics
from sklearn.metrics import confusion_matrix
r=[1 if i>0.6 else 0 for i in oof_xgb1]
print('confusion_matrix for lr:\n', metrics.confusion_matrix(my_xx1['label'].values,r))
print('classification_report for lr\n:', metrics.classification_report(my_xx1['label'].values,r))
print('到这里就结束了！下面的cell可以不用管！下面保存特征：')
stacking_tr=my_xx1[['id']]
stacking_tr['pre']=oof_xgb1
stacking_te=my_te[['id']]
stacking_te['pre']=prediction_xgb1
stacking_tr.to_csv('stacking_tr.csv',index=None) ##训练集stacking特征
stacking_te.to_csv('stacking_te.csv',index=None) ##测试集stacking特征


fold: 1 training
[0]	train-auc:0.99045	valid_data-auc:0.987603
Multiple eval metrics have been passed: 'valid_data-auc' will be used for early stopping.

Will train until valid_data-auc hasn't improved in 200 rounds.
[400]	train-auc:0.998736	valid_data-auc:0.996656
[800]	train-auc:0.999837	valid_data-auc:0.998267
Stopping. Best iteration:
[876]	train-auc:0.99989	valid_data-auc:0.998288

fold: 2 training
[0]	train-auc:0.990545	valid_data-auc:0.986774
Multiple eval metrics have been passed: 'valid_data-auc' will be used for early stopping.

Will train until valid_data-auc hasn't improved in 200 rounds.
[400]	train-auc:0.998655	valid_data-auc:0.9963
[800]	train-auc:0.999864	valid_data-auc:0.998187
[1200]	train-auc:0.999986	valid_data-auc:0.998209
Stopping. Best iteration:
[1065]	train-auc:0.99997	valid_data-auc:0.998235

fold: 3 training
[0]	train-auc:0.989163	valid_data-auc:0.990338
Multiple eval metrics have been passed: 'valid_data-auc' will be used for early stopping.

Will train unti

"\ntrain_B_info=my_xx1.describe()\nmeaningful_col=[]\nfor col in train_B_info.columns:\n    if train_B_info.ix[0,col]>my_xx1.shape[0]*0.01:#ix是用了索引指定的数据，ix[0,col]为0行col列的数;shape[0]返回行数\n        meaningful_col.append(col)#将col放到meaningful_col最后一列中\nmy_xx1=my_xx1[meaningful_col].copy()#copy to train_B_1\nmeaningful_col.remove('label')\nmy_te1=my_te1[meaningful_col].copy()\nprint(meaningful_col)\n\n#oof_xgb1,prediction_xgb1=xgb_model(np.array(my_xx1.drop(['id','label'],axis=1)),#                       my_xx1['label'].values,#                       np.array(my_te1.drop(['id'],axis=1)),lr=0.1)\noof_xgb1,prediction_xgb1=xgb_model(np.array(my_xx[meaningful_col+['id']]),                       my_xx['label'].values,                       np.array(my_te[meaningful_col+['id']]),0.01)\n#oof_xgb1,prediction_xgb1,impo=lgb_model(np.array(my_xx1.drop(['id','label'],axis=1)),#                       my_xx1['label'].values,#                       np.array(my_te1.drop(['id'],axis=1)))\n\n#result1=my_xx1[[

time: 23min 13s


求分错的用户是哪些用户...

In [186]:

from sklearn import metrics
from sklearn.metrics import confusion_matrix
r=[1 if i>0.5 else 0 for i in oof_xgb1]
print('confusion_matrix for lr:\n', metrics.confusion_matrix(my_xx1['label'],r))
print('classification_report for lr\n:', metrics.classification_report(my_xx1['label'],r))

##分错的用户存于cuo_id中：字段含义为：用户id,真实标签label,模型预测概率：
cuo_id=[]
for i in range(len(r)):
    if r[i]!=my_xx1['label'][i]:
        cuo_id.append([my_xx1.loc[i,'id'],my_xx1['label'][i],oof_xgb1[i]])
cuo_id=pd.DataFrame(cuo_id,columns=['id','label','pre'])
print(cuo_id)

confusion_matrix for lr:
 [[60882     0]
 [  136  3516]]
classification_report for lr
:               precision    recall  f1-score   support

           0       1.00      1.00      1.00     60882
           1       1.00      0.96      0.98      3652

    accuracy                           1.00     64534
   macro avg       1.00      0.98      0.99     64534
weighted avg       1.00      1.00      1.00     64534

time: 1.03 s


In [187]:
'''
##这里可以不用管：
print(cuo_id)
#print(new_tr[new_tr['id'].isin([int(i) for i in cuo_id['id']])])
all_id=list(set(q1['id']))
q6_id=list(set(q6['id']))
not_in_q6_id=[i for i in all_id if i not in q6_id]
print('not_in_q6_id:',len(not_in_q6_id))
#print(q1.nunique())
#print(cuo_id[:10])
#print(q1['id'][:10])
print(len(set(q7[q7['id'].isin([int(i) for i in cuo_id['id']])]['id'])))
print(len(set(n6['id'].unique())&set(n7['id'].unique())))
print(len(set(q6['id'].unique())&set(q1['id'].unique())))
print(len(set(not_in_q6_id)&set(q4['id'].unique())))
#print(q3[q3['id'].isin([int(i) for i in cuo_id['id']])].sort_values(by=['id'],ascending=True))
#print(q2[:200])
mmm=list(n3.sort_values(by=['id'],ascending=True).index)
zzz=[]
for i in range(0,11200-1,2):
    zzz.append(abs(mmm[i+1]-mmm[i]))
zzz=pd.Series(zzz)
print(zzz.value_counts())

mmm=list(q1.sort_values(by=['id'],ascending=True).index)
zzz=[]
for i in range(0,11200-1,2):
    zzz.append([q1.loc[mmm[i+1],'id'],abs(mmm[i+1]-mmm[i])])
zzz=pd.DataFrame(zzz,columns=['id','juli'])
print(zzz.sort_values(by=['id'],ascending=True))
print(zzz[zzz.juli<4500].shape)
leak_id=list(zzz[zzz.juli<4500]['id'])
#print(pd.DataFrame(zzz['juli'].value_counts()))
#guo=pd.DataFrame(zzz['juli'].value_counts())
#print(guo[guo.juli<2].shape)
#print(q1[q1.id.isin([int(i) for i in cuo_id['id']])].sort_values(by=['id'],ascending=True))
'''

                   id  label       pre
0    1837472551960900      1  0.027038
1    4209653910405500      1  0.044190
2    1449939502334500      1  0.036933
3    4396279590290000      1  0.054725
4    7217048611554200      1  0.005852
5    8724124477031600      1  0.151961
6    1886006687304000      1  0.023341
7    9447563464608100      1  0.040569
8    1468333456263700      1  0.004428
9    8537829126507120      1  0.103902
10   4275873842793200      1  0.010385
11   6413063328533255      1  0.007890
12   8090377968520000      1  0.025474
13   8605746201759424      1  0.009946
14   6518101911965700      1  0.219370
15   8024934942806133      1  0.034839
16   2139805723627900      1  0.215362
17   6497562401772428      1  0.054449
18   4538334245033000      1  0.063827
19   5260608249861300      1  0.006627
20   1742729033422100      1  0.025594
21   2613895006393700      1  0.096620
22   8400462932678237      1  0.098320
23   5175939607729779      1  0.171622
24   2571335048926900    

In [188]:
'''
w=list(set(test2['id']))
w1=list(test['id'])
not_in_test2=[i for i in w1 if i not in w]
w2=list(set(test6['id']))
not_2_6=[i for i in not_in_test2 if i not in w2]
print('不在表2和6的用户数',len(not_2_6))
#print(len(not_in_test2))
#print(feature_importance_df.sort_values(by='importance',ascending=False))
#print(test5.shape,test5.nunique())
#print(f5(test5)[:100])
#print(n5.shape,n5.nunique())
#print(q5.shape,q5.nunique())
#w=train[['id','label','6是否去过黔东南目标景区',\
#'7是否去过黔东南目标景区']]
#w1=test[['id','6是否去过黔东南目标景区',\
#'7是否去过黔东南目标景区']]
#w=pd.merge(w,w1,how='left',on=['id'])
#w.columns=['id','y','6','7','66','77']
#print(w.sort_values(by='77',ascending=True))
#print(w1.sort_values(by='id',ascending=False))

#print(test[test.id.isin(list(xiu2.id))][['id','6是否去过黔东南目标景区',\
#'7是否去过黔东南目标景区']]['7是否去过黔东南目标景区'].value_counts())

sub=test[['id']].astype(str)
sub['Pred']=prediction_xgb1
#sub=sub[~sub.id.isin([str(i) for i in list(xiu2.id)])].reset_index(drop=True)
#uid6=list(te['id'])
#print(len(uid6))
#print(len([i for i in uid6 if i not in null_id]))
#print(len(list(set(test1['id']))))
sub['Pred']=sub['Pred'].apply(lambda x:round(x,8))
sub.columns=['ID','Pred']
#sub.loc[sub.ID.isin([str(i) for i in leak_id]),'Pred']=1.0000#0.0000
#sub.loc[sub.ID.isin([str(i) for i in list(idss)]),'Pred']=0.0000
sub.to_csv('mysubmission.csv',index=None)
print(sub['Pred'].describe())
print(len(sub[sub.Pred>0.6]))
print(len(sub[sub.Pred>0.0]))
print(sub[sub.ID.isin([str(i) for i in not_in_test2])]['Pred'].describe())
#print(sub[sub.ID.isin([str(i) for i in null_id if i not in null_id2])]['Pred'].describe())
res_id=list(sub[sub.Pred>0.6]['ID'])
#print(zzz[zzz.id.isin([int(i) for i in res_id])]['juli'].value_counts())
'''

不在表2和6的用户数 1532
count    50200.000000
mean         0.151749
std          0.296717
min          0.001349
25%          0.001949
50%          0.002431
75%          0.104898
max          0.998063
Name: Pred, dtype: float64
4296
50200
count    1532.000000
mean        0.478976
std         0.016612
min         0.008491
25%         0.481293
50%         0.481293
75%         0.481293
max         0.561069
Name: Pred, dtype: float64


NameError: name 'ed' is not defined

time: 35 s


In [ ]:
#!wget -nv -O kesci_submit https://www.heywhale.com/kesci_submit&&chmod +x kesci_submit
#!wget -O kesci_submit https://www.heywhale.com/kesci_submit&&chmod +x kesci_submit

In [189]:
##提交结果：


Kesci Submit Tool 3.0

> 已验证Token
> 提交文件 mysubmission.csv (1366.88 KiB)
> 文件已上传        
> 提交完成
time: 2.17 s
